# 실행하기 전에
- content에와 같은 경로에 test폴더 생성 후, test 폴더 안에 images, masks 폴더 생성

```
content
test
  ㄴ images
  ㄴ masks
```



# Data load and patch

In [ ]:
!git clone https://github.com/louisyuzhe/car-damage-detector.git

Cloning into 'car-damage-detector'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 163 (delta 8), reused 27 (delta 4), pack-reused 131
Receiving objects: 100% (163/163), 83.39 MiB | 18.35 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
# !pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

# Import

In [ ]:
cd /content/car-damage-detector

/content/car-damage-detector


In [ ]:
#Import modules
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import skimage.draw

import shutil

# Import Mask RCNN
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
# from mrcnn import model
# import mrcnn.model as modellib
# from mrcnn.model import log
import cv2
import custom
import imgaug,h5py,IPython

%matplotlib inline

# Root directory of the project

In [ ]:
ROOT_DIR = os.getcwd()
sys.path.append(ROOT_DIR)  # To find local version of the library

# Configurations
config = custom.CustomConfig()
dataset_DIR = os.path.join(ROOT_DIR, "")
dataset_DIR

'/content/car-damage-detector/'

# Load dataset and Data check

In [ ]:
dataset = custom.CustomDataset()
dataset.load_custom(dataset_DIR, "train")
dataset.load_custom(dataset_DIR, "val")
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

Image Count: 64
Class Count: 2
  0. BG                                                
  1. scratch                                           


# Display function

In [ ]:
def display_images(images, titles=None, cols=0, cmap=None, norm=None,
                   interpolation=None):
    """Display the given set of images, optionally with titles.
    images: list or array of image tensors in HWC format.
    titles: optional. A list of titles to display with each image.
    cols: number of images per row
    cmap: Optional. Color map to use. For example, "Blues".
    norm: Optional. A Normalize instance to map values to colors.
    interpolation: Optional. Image interpolation to use for display.
    """
    titles = titles if titles is not None else [""] * len(images)
    rows = len(images) // cols + 1
    plt.figure(figsize=(14, 14 * rows // cols))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.title(title, fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                   norm=norm, interpolation=interpolation)
        i += 1
    plt.show()

In [ ]:
def display_top_masks(image, mask, class_ids, class_names, limit=1):
    """Display the given image and the top few class masks."""
    to_display = []
    titles = []
    # to_display.append(image)
    # titles.append("H x W={}x{}".format(image.shape[0], image.shape[1]))
    # Pick top prominent classes in this image
    unique_class_ids = np.unique(class_ids)
    mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]])
                 for i in unique_class_ids]
    top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),
                                    key=lambda r: r[1], reverse=True) if v[1] > 0]
    # Generate images and titles
    for i in range(limit):
        class_id = top_ids[i] if i < len(top_ids) else -1
        # Pull masks of instances belonging to the same class.
        m = mask[:, :, np.where(class_ids == class_id)[0]]
        m = np.sum(m * np.arange(1, m.shape[-1] + 1), -1)
        to_display.append(m)
        titles.append(class_names[class_id] if class_id != -1 else "-")
    display_images(to_display, titles=None, cols=limit + 1, cmap="Greys_r")

    return to_display, titles

In [ ]:
# dataset.image_info

# Image check and save

In [ ]:
# img_path = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/1. MAIN/6. Data/a_new_data_0627/train/'

In [ ]:
img_path = '/content/test/'

In [ ]:
# Load and display mask samples 
image_ids = dataset.image_ids

file_name_list = []
file_name_only = []
for image_id in image_ids:
    file_names = dataset.image_info[image_id]['id']
    file_name_list.append(file_names)
    file_name = file_names.split('.')[0]
    file_name_only.append(file_name)
    print(file_name)
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    a, b = display_top_masks(image, mask, class_ids, dataset.class_names)

# mask image save
    plt.imsave(img_path+f'masks/a_{file_name}.jpg', a[0], cmap = 'gray')

# Image move

In [ ]:
before_path_train = '/content/car-damage-detector/dataset/train/'
before_path_valid = '/content/car-damage-detector/dataset/val/'
move_path = '/content/test/images/'

In [ ]:
for name, only in zip(file_name_list, file_name_only):
    # print(test_path + name)
    try:
        shutil.move(before_path_train + name, move_path+f'/a_{only}.jpg')
    except:
        shutil.move(before_path_valid + name, move_path+f'/a_{only}.jpg')    

# Data zip

In [ ]:
cd /content/test

/content/test


In [ ]:
!zip -r a_images.zip ./images
!zip -r a_mask.zip ./masks

# Practice

In [ ]:
a_path = '/content/car-damage-detector/dataset/train'
b_path = '/content/car-damage-detector/dataset/val'

In [ ]:
a_dir = os.listdir(a_path)
b_dir = os.listdir(b_path)

In [ ]:
all_dir = []
all_path = []

In [ ]:
all_dir

In [ ]:
for dir in a_dir:
    if dir[-4:] != 'json':
        all_dir.append(dir.split('.')[0])
        all_path.append(a_path+'/'+dir)

In [ ]:
for dir in b_dir:
    if dir[-4:] != 'json':
        all_dir.append(dir.split('.')[0])
        all_path.append(b_path+'/'+dir)

In [ ]:
for a, b in zip(all_path, all_dir):
    print(a, b)

In [ ]:
import shutil

In [ ]:
for path, dir in zip(all_path, all_dir):
    shutil.move(path, img_path+f'images/a_{dir}.jpg')

In [ ]:
img_paths = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/1. MAIN/6. Data/a_new_data_0627/train/masks'

In [ ]:
a_dir = os.listdir(img_paths)
a_dir

In [ ]:
for a, all in zip(a_dir, all_dir):
    img_path = img_paths + '/' + a
    shutil.move(img_path, img_paths+f'/a_{all}.jpg')